In [2]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import wandb

idle_time_data = pd.read_csv('../data/df_points/df_points_18_21_class.csv')
TargetVariable = ['idle_time_class']
Predictors = ['bike_id', 'lat', 'lng', 'temp', 'rain', 'snow', 'dt_start', 'hex_enc', 'start_min', 'month', 'day']

X = idle_time_data[Predictors].values
y = idle_time_data[TargetVariable].values

PredictorScaler=StandardScaler()
PredictorScalerFit=PredictorScaler.fit(X)
X=PredictorScalerFit.transform(X)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9)
# , random_state=42

In [ ]:
from sklearn.metrics import zero_one_loss, accuracy_score, roc_auc_score, f1_score
from sklearn.model_selection import cross_validate, cross_val_score, GridSearchCV
from sklearn.svm import SVC
import wandb
import wandb

sweep_configuration_rfc = {
    "project": "SVC",
    "name": "my-awesome-sweep",
    "metric": {"name": "accuracy", "goal": "maximize"},
    "method": "random",
    "parameters": {
        "C": {
            "values": [0.6, 0.8, 1.0, 1.2, 1.4]
        },
        "kernel": {
            "values": ['linear','poly','rbf','sigmoid','precomputed']
        },
        "degree": {
            "values": [2,3,4,5]
        },
        "gamma": {
            "values": ['scale', 'auto']
        },
        "decision_function_shape": {
            "values": ['ovo']
        }
    }
}

def my_train_func():
    wandb.init()

    _C = wandb.config.C
    _kernel = wandb.config.kernel
    _degree = wandb.config.degree
    _gamma = wandb.config.gamma
    _decision_function_shape=wandb.config.decision_function_shape

    model = SVC(C=_C,
                kernel=_kernel,
                degree=_degree,
                gamma=_gamma,
                decision_function_shape=_decision_function_shape
                )

    model.fit(X_train, y_train.ravel())
    y_pred = model.predict(X_test)

    print("roc_auc_score: ", roc_auc_score(y_test.ravel(), y_pred.ravel()))
    print("f1 score: ", f1_score(y_test.ravel(), y_pred.ravle()))

    acc = zero_one_loss(y_test.ravel(), y_pred.ravel())
    loss = accuracy_score(y_test.ravel(), y_pred.ravel())

    #wandb.sklearn.plot_feature_importances(model, Predictors)

    wandb.log({"accuracy": acc})
    wandb.log({"conf_matrix": wandb.plot.confusion_matrix(y_true=y_test.ravel(), preds=y_pred.ravel())})
    wandb.log({"loss": loss})


# INIT SWEEP
sweep_id_svc = wandb.sweep(sweep_configuration_rfc, project="SVC")
# RUN SWEEP
wandb.agent(sweep_id_svc, function=my_train_func)

Error in callback <function _WandbInit._resume_backend at 0x7f7dd8b1bf70> (for pre_run_cell):


Exception: The wandb backend process has shutdown

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: 3t84qu5g
Sweep URL: https://wandb.ai/jonathanweske/SVC/sweeps/3t84qu5g


wandb: Agent Starting Run: 543xfnv4 with config:
wandb: 	C: 0.6
wandb: 	decision_function_shape: ovo
wandb: 	degree: 3
wandb: 	gamma: auto
wandb: 	kernel: sigmoid
